In [1]:
# !pip install selenium schedule

In [ ]:
import os
import time
import datetime
import schedule
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# ✅ ChromeDriver path — MODIFY this only if needed
CHROMEDRIVER_PATH = r"D:\VSCode\Projects\Firewall-LogFile_Automate\chromedriver.exe"

# ✅ Daily download folder base path
BASE_DOWNLOAD_PATH = r"D:\F-Log report\LOG"

# ✅ Create daily folder (after 5 AM)
def create_daily_folder():
    now = datetime.datetime.now()
    date_folder = (now if now.hour >= 5 else now - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    folder_path = os.path.join(BASE_DOWNLOAD_PATH, date_folder)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

# ✅ Setup Chrome WebDriver
def get_driver(download_path):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--allow-insecure-localhost")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")

    # 👇 REMOVE this line to make browser visible
    # chrome_options.add_argument("--headless=new")  # <-- Commented out for visible browser

    # Bypass certificate warning
    chrome_options.add_argument("--allow-running-insecure-content")

    # Set download path
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "safebrowsing.enabled": True
    })

    return webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=chrome_options)

# ✅ Main automation logic
def auto_login_and_download():
    download_dir = create_daily_folder()
    driver = get_driver(download_dir)
    wait = WebDriverWait(driver, 20)

    try:
        print(f"[{datetime.datetime.now()}] 🌐 Opening portal...")
        driver.get("https://10.0.100.62:4444")

        # Login
        wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/form/div/div/div[2]/div[1]/div[1]/div[1]/input[1]"))).send_keys("admin")
        driver.find_element(By.XPATH, "/html/body/form/div/div/div[2]/div[1]/div[1]/div[1]/input[2]").send_keys("Ipac#2023#@!253")
        driver.find_element(By.XPATH, "/html/body/form/div/div/div[2]/div[1]/div[1]/div[1]/input[3]").click()

        print("✅ Logged in successfully.")

        # Navigate to report
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section[1]/div[3]/nav/section[1]/ul/li[3]/a"))).click()
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section[2]/div[1]/div[3]/div/nav/ul/li[5]"))).click()
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[1]/div[1]/div[2]/label[1]"))).click()

        # Select report filters
        driver.find_element(By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[2]/div/table/tbody/tr[1]/td[2]/label[2]").click()
        driver.find_element(By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[2]/div/table/tbody/tr[2]/td[2]/div[1]/label").click()
        driver.find_element(By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[2]/div/table/tbody/tr[3]/td[2]/label[1]").click()

        # Click search and download
        driver.find_element(By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[1]/div/div/form/div[2]/div/table/tbody/tr[10]/td[2]/input").click()
        wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/section[2]/div[2]/div[2]/div[2]/div[2]/div[6]/div/div/label"))).click()

        print(f"⬇️  Report download started. Saving to: {download_dir}")
        time.sleep(10)

    except TimeoutException as te:
        print("⛔ Timeout error:", te)
    except Exception as e:
        print("⛔ Unexpected error:", e)
    finally:
        driver.quit()
        print("✅ Task complete. Browser closed.")

# ✅ Schedule to run every 20 minutes
schedule.every(20).minutes.do(auto_login_and_download)

print("🕒 Scheduler started. Task will repeat every 20 minutes...")

# ✅ Infinite loop to keep the scheduler alive
while True:
    schedule.run_pending()
    time.sleep(1)
